In [89]:
API_KEY = "fa25af98-8aed-443e-9803-b848e9912836"


In [88]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import os




In [90]:
url = 'https://content.guardianapis.com/search?q=fifa%20AND%20qatar&from-date=2010-01-01&api-key=fa25af98-8aed-443e-9803-b848e9912836&page='

In [91]:
urllist=[]


In [92]:
for i in range(1, 179):
    base_url = 'https://content.guardianapis.com/search?q=fifa%20AND%20qatar&from-date=2010-01-01&api-key=fa25af98-8aed-443e-9803-b848e9912836&page='
    url = f"{base_url}{i}"
    urllist.append(url)

In [93]:
info=[]

In [94]:
def json(url1):
    response=requests.get(url1)
    x=response.json()
    info.append(x)

In [95]:
output=[json(url1) for url1 in urllist]


In [96]:
finallist=[]


In [97]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'
}

for y in range(1, 178):
    for j in range(0, 10):
        url = info[y]['response']['results'][j]['webUrl']
        with requests.Session() as session:
                response = session.get(url, headers=headers)
                #time.sleep(random.uniform(1, 3))  # Add random delay
                soup = BeautifulSoup(response.content, 'html.parser')
                full_text = ""
                for element in soup.find_all(['p']):  # Adjust tags accordingly
                    full_text += element.get_text(separator="\n")
                published_date = info[y]['response']['results'][j]['webPublicationDate']
                # Parse published_date and extract day, month, year
                published_datetime = datetime.strptime(published_date, "%Y-%m-%dT%H:%M:%SZ")
                day = published_datetime.day
                month = published_datetime.month
                year = published_datetime.year

                value = {
                    'webtitle': info[y]['response']['results'][j]['webTitle'],
                    'apiUrl':info[y]['response']['results'][j]['webUrl'],
                    'publisheddate': f"{day:02d}.{month:02d}.{year}",  # Format as numbers
                    'Text': full_text
                }
                if value not in finallist:
                    finallist.append(value)
  
            

In [99]:
unique_list_of_dicts = [dict(t) for t in {tuple(d.items()) for d in finallist}]


In [100]:
datanew=pd.DataFrame(unique_list_of_dicts)
datanew

,webtitle,apiUrl,publisheddate,Text
0,Revealed: star football pundits promote World ...,https://www.theguardian.com/football/2022/nov/...,20.11.2022,Spirit of rules on use of celebrities in adver...
1,Fifa ethics chief Michael Garcia calls for gre...,https://www.theguardian.com/football/2014/oct/...,13.10.2014,Garcia has delivered his report on 2018 and 2...
2,World Cup 2022: news and buildup as the knocko...,https://www.theguardian.com/football/live/2022...,03.12.2022,Brazil suffer double injury blow going into th...
3,England’s Harry Kane may abandon ‘OneLove’ arm...,https://www.theguardian.com/football/2022/nov/...,20.11.2022,England’s players will take the knee before th...
4,Welcome to Lusail: Lego-city of the gods and o...,https://www.theguardian.com/football/2022/nov/...,27.11.2022,Andy Warhol would love the venue for the World...
...,...,...,...,...
1751,UK parliament calls on Fifa to support exiled ...,https://www.theguardian.com/football/2023/jun/...,20.06.2023,More than 100 parliamentarians have written to...
1752,Jesús Ferreira scores four as USA breeze past ...,https://www.theguardian.com/football/2022/jun/...,11.06.2022,Jesús Ferreira tied the United States’ interna...
1753,"The Guide #29: From All 4 to Netflix, the one ...",https://www.theguardian.com/culture/2022/apr/0...,08.04.2022,In this week’s newsletter: not sure what to st...
1754,World Cup 2022 briefing: Louis van Gaal is a m...,https://www.theguardian.com/football/2022/dec/...,09.12.2022,Louis van Gaal’s Dutch team may not play beaut...


In [101]:
csv_file = "/Users/tarek/Desktop/Data.csv"
# Customize the export settings
custom_header = ['webtitle','apiUrl','publisheddate','Text']

datanew.to_csv(csv_file, index=False, na_rep='N/A', header=custom_header, sep=',', quoting=csv.QUOTE_NONNUMERIC)


In [ ]:
csv_file = "/Users/tarek/Desktop/Names.xlsx"
# Customize the export settings
custom_header = ['webtitle','apiUrl','publisheddate','Text']

datanew.to_excel(csv_file, index=False, na_rep='N/A', header=custom_header, index_label='ID')